# 고급 RAG 기법 (Advanced RAG Techniques)

이번 노트북에서는 **고급 RAG 기법**을 학습합니다.

## 학습 목표
기본 RAG를 넘어서 실무에서 활용할 수 있는 고급 기법들을 익힙니다.

## 오늘 배울 내용: Ingest (데이터 수집 및 전처리) 고급 기법

1. **LangChain 없이 순수 Python으로 구현** - 최대한의 유연성을 위해 프레임워크 의존성 제거
2. **LLM을 활용한 지능형 청킹** - 단순 문자 수 기반이 아닌, LLM이 문맥을 이해하여 의미 있는 단위로 분할
3. **최적의 청크 크기와 임베딩 모델 사용** - 이전 실습에서 찾은 최적의 설정 활용
4. **문서 전처리 (Document Pre-processing)** - LLM이 검색에 유리하도록 청크를 재작성

## 왜 고급 RAG가 필요한가?

기본 RAG의 한계:
- 고정 크기 청킹으로 인한 문맥 손실
- 단순 유사도 검색의 정확도 한계
- 사용자 질문과 문서 표현 방식의 불일치

고급 RAG의 해결책:
- **Semantic Chunking**: 의미 단위로 문서 분할
- **Reranking**: 검색 결과 재정렬로 정확도 향상
- **Query Rewriting**: 검색에 최적화된 쿼리로 변환

In [ ]:
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from chromadb import PersistentClient
from tqdm import tqdm
from litellm import completion
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go


load_dotenv(override=True)

MODEL = "gpt-4.1-nano"

DB_NAME = "preprocessed_db"
collection_name = "docs"
embedding_model = "text-embedding-3-large"
KNOWLEDGE_BASE_PATH = Path("knowledge-base")
AVERAGE_CHUNK_SIZE = 500

openai = OpenAI()

In [ ]:
# LangChain의 Document 클래스에서 영감을 받아 유사한 구조 정의

class Result(BaseModel):
    page_content: str
    metadata: dict

In [ ]:
# Chunk를 표현하는 Pydantic 모델
# LLM의 Structured Output을 위해 Field description을 상세히 작성

class Chunk(BaseModel):
    headline: str = Field(description="A brief heading for this chunk, typically a few words, that is most likely to be surfaced in a query")
    summary: str = Field(description="A few sentences summarizing the content of this chunk to answer common questions")
    original_text: str = Field(description="The original text of this chunk from the provided document, exactly as is, not changed in any way")

    def as_result(self, document):
        metadata = {"source": document["source"], "type": document["type"]}
        return Result(page_content=self.headline + "\n\n" + self.summary + "\n\n" + self.original_text,metadata=metadata)


class Chunks(BaseModel):
    chunks: list[Chunk]

## 데이터 수집 파이프라인 3단계

RAG 시스템의 데이터 수집(Ingest) 과정은 다음 3단계로 구성됩니다:

1. **문서 로드 (Fetch)** - Knowledge Base에서 문서들을 가져옵니다 (LangChain의 DirectoryLoader와 유사)
2. **LLM 기반 청킹** - LLM을 호출하여 문서를 의미 있는 Chunk로 분할합니다
3. **벡터 저장** - 생성된 Chunk들을 ChromaDB에 저장합니다

이것이 전부입니다! 간단하죠?

### Step 1: 문서 로드하기

아래 함수는 LangChain의 `DirectoryLoader`를 순수 Python으로 구현한 것입니다.
폴더 구조에서 문서 타입을 추출하고, 모든 마크다운 파일을 읽어옵니다.

In [ ]:
def fetch_documents():
    """LangChain DirectoryLoader의 순수 Python 구현"""

    documents = []

    for folder in KNOWLEDGE_BASE_PATH.iterdir():
        doc_type = folder.name
        for file in folder.rglob("*.md"):
            with open(file, "r", encoding="utf-8") as f:
                documents.append({"type": doc_type, "source": file.as_posix(), "text": f.read()})

    print(f"Loaded {len(documents)} documents")
    return documents

In [ ]:
documents = fetch_documents()

In [ ]:
# 첫 번째 문서의 구조 확인
print(f"문서 개수: {len(documents)}")
print(f"\n첫 번째 문서 구조:")
print(f"  - type: {documents[0]['type']}")
print(f"  - source: {documents[0]['source']}")
print(f"  - text 길이: {len(documents[0]['text'])} 글자")
print(f"  - text 미리보기: {documents[0]['text'][:200]}...")

### 문서(Document) 구조 이해하기

`fetch_documents()` 함수가 반환하는 `documents`는 딕셔너리 리스트입니다.

**각 문서의 구조:**

```python
{
    "type": str,    # 문서 타입 (폴더 이름에서 추출)
    "source": str,  # 파일 경로 (출처 추적용)
    "text": str     # 파일 전체 내용
}
```

**Knowledge Base 폴더 구조 예시:**

```
knowledge-base/
├── products/           # type = "products"
│   ├── product1.md
│   └── product2.md
├── employees/          # type = "employees"
│   ├── ceo.md
│   └── cto.md
├── contracts/          # type = "contracts"
│   └── contract1.md
└── company/            # type = "company"
    └── about.md
```

**왜 이 구조인가?**

| 필드 | 용도 |
|-----|------|
| `type` | 문서 분류, 필터링 검색, 시각화 색상 구분에 활용 |
| `source` | 답변 시 출처 표시, 디버깅, 추적에 활용 |
| `text` | LLM 청킹의 입력 데이터 |

이 메타데이터는 나중에 ChromaDB에 저장되어 검색 결과와 함께 반환됩니다.

### Step 2: LLM 기반 지능형 청킹

이제 핵심인 **LLM 기반 청킹**을 구현합니다.

기존 방식 vs LLM 기반 방식:

| 기존 방식 | LLM 기반 방식 |
|----------|--------------|
| 고정 문자 수로 분할 | 의미 단위로 분할 |
| 문맥 무시 | 문맥 이해 |
| 단순 오버랩 | 지능적 오버랩 |
| 원본 텍스트만 저장 | headline + summary + 원본 저장 |

LLM에게 다음을 요청합니다:
- **headline**: 검색에 유리한 짧은 제목
- **summary**: 핵심 내용 요약
- **original_text**: 원본 텍스트 (변경 없이)

이렇게 하면 검색 시 headline과 summary가 매칭 확률을 높여줍니다.

In [ ]:
def make_prompt(document):
    how_many = (len(document["text"]) // AVERAGE_CHUNK_SIZE) + 1
    return f"""
You take a document and you split the document into overlapping chunks for a KnowledgeBase.

The document is from the shared drive of a company called Insurellm.
The document is of type: {document["type"]}
The document has been retrieved from: {document["source"]}

A chatbot will use these chunks to answer questions about the company.
You should divide up the document as you see fit, being sure that the entire document is returned in the chunks - don't leave anything out.
This document should probably be split into {how_many} chunks, but you can have more or less as appropriate.
There should be overlap between the chunks as appropriate; typically about 25% overlap or about 50 words, so you have the same text in multiple chunks for best retrieval results.

For each chunk, you should provide a headline, a summary, and the original text of the chunk.
Together your chunks should represent the entire document with overlap.

Here is the document:

{document["text"]}

Respond with the chunks.
"""

In [ ]:
print(make_prompt(documents[0]))

In [ ]:
def make_messages(document):
    return [
        {"role": "user", "content": make_prompt(document)},
    ]

In [ ]:
make_messages(documents[0])

### process_document 함수 상세 설명

`process_document`는 이 노트북의 **핵심 함수**입니다. 단일 문서를 LLM에게 전달하여 지능형 청킹을 수행합니다.

**함수 동작 흐름:**

```
문서 (Document)
    ↓
make_messages() - 프롬프트 생성
    ↓
LLM API 호출 (litellm.completion)
    ↓
Structured Output (JSON) 파싱
    ↓
Chunk 객체 리스트로 변환
    ↓
Result 객체 리스트 반환
```

**핵심 포인트:**

1. **Structured Output 활용**
   - `response_format=Chunks`를 지정하여 LLM이 정해진 JSON 스키마로 응답
   - Pydantic 모델로 자동 검증 및 파싱

2. **LLM이 결정하는 것들**
   - 청크 분할 위치 (의미 단위로)
   - 각 청크의 headline (검색용 제목)
   - 각 청크의 summary (핵심 요약)
   - 오버랩 범위 (문맥 보존을 위해)

3. **왜 이 방식이 효과적인가?**
   - 단순 문자 수 분할은 문장 중간에서 끊길 수 있음
   - LLM은 문단, 주제, 논리적 구분을 이해함
   - headline과 summary가 검색 매칭률을 높임

In [ ]:
def process_document(document):
    """
    단일 문서를 LLM 기반 지능형 청킹으로 처리하는 핵심 함수
    
    Args:
        document: dict - {"type": str, "source": str, "text": str} 형태의 문서
        
    Returns:
        list[Result] - headline, summary, original_text가 결합된 Result 객체 리스트
        
    처리 과정:
        1. make_messages()로 LLM 프롬프트 생성
        2. litellm.completion()으로 LLM 호출 (Structured Output 사용)
        3. Pydantic으로 JSON 응답을 Chunks 모델로 파싱
        4. 각 Chunk를 Result 객체로 변환하여 반환
    """
    # 1. 문서를 LLM 메시지 형식으로 변환
    messages = make_messages(document)
    
    # 2. LLM 호출 - response_format으로 Pydantic 모델을 지정하면
    #    LLM이 해당 스키마에 맞는 JSON으로 응답함 (Structured Output)
    response = completion(model=MODEL, messages=messages, response_format=Chunks)
    
    # 3. LLM 응답에서 JSON 문자열 추출
    reply = response.choices[0].message.content
    
    # 4. JSON을 Pydantic 모델로 파싱하여 Chunk 리스트 획득
    doc_as_chunks = Chunks.model_validate_json(reply).chunks
    
    # 5. 각 Chunk를 Result 형태로 변환 (headline + summary + original_text 결합)
    return [chunk.as_result(document) for chunk in doc_as_chunks]

In [ ]:
process_document(documents[0])

In [ ]:
# tqdm: 진행률 표시 라이브러리
# - 반복문을 tqdm()으로 감싸면 자동으로 프로그레스 바 표시
# - LLM API 호출은 시간이 오래 걸리므로 진행 상황 확인에 유용
# - 예상 남은 시간(ETA), 처리 속도(it/s) 등도 함께 표시됨

def create_chunks(documents):
    """모든 문서를 순회하며 LLM 기반 청킹 수행"""
    chunks = []
    for doc in tqdm(documents):  # tqdm(documents): 진행률 바와 함께 반복
        chunks.extend(process_document(doc))
    return chunks

In [ ]:
chunks = create_chunks(documents)

**tqdm 출력 예시:**
```
100%|██████████| 25/25 [01:30<00:00,  3.62s/it]
```

| 표시 | 의미 |
|-----|------|
| `100%` | 진행률 |
| `██████████` | 프로그레스 바 |
| `25/25` | 완료/전체 개수 |
| `01:30` | 경과 시간 |
| `<00:00` | 예상 남은 시간 |
| `3.62s/it` | 항목당 처리 시간 |

In [ ]:
print(len(chunks))

### Step 2 완료! 

LLM 기반 청킹이 완료되었습니다. 다소 시간이 걸리지만, 결과의 품질은 훨씬 좋습니다.

**성능 최적화 팁:**
- 실제 프로덕션에서는 `multiprocessing.Pool`을 사용해 병렬 처리할 수 있습니다
- Rate Limit 오류가 발생하면 병렬 처리를 끄거나 지연 시간을 추가하세요
- 배치 처리로 API 호출 횟수를 줄일 수도 있습니다

### Step 3: 임베딩 생성 및 벡터 DB 저장

이제 생성된 청크들을 벡터로 변환하여 ChromaDB에 저장합니다.

**이 단계에서 일어나는 일:**
1. 기존 컬렉션이 있으면 삭제 (재생성)
2. 모든 청크 텍스트를 임베딩 벡터로 변환
3. ChromaDB 컬렉션에 벡터, 문서, 메타데이터 저장

In [ ]:
def create_embeddings(chunks):
    chroma = PersistentClient(path=DB_NAME)
    if collection_name in [c.name for c in chroma.list_collections()]:
        chroma.delete_collection(collection_name)

    texts = [chunk.page_content for chunk in chunks]
    emb = openai.embeddings.create(model=embedding_model, input=texts).data
    vectors = [e.embedding for e in emb]

    collection = chroma.get_or_create_collection(collection_name)

    ids = [str(i) for i in range(len(chunks))]
    metas = [chunk.metadata for chunk in chunks]

    collection.add(ids=ids, embeddings=vectors, documents=texts, metadatas=metas)
    print(f"Vectorstore created with {collection.count()} documents")

In [ ]:
create_embeddings(chunks)

# 벡터 시각화

잠깐! 시각화를 빼먹을 뻔했네요!

벡터 데이터베이스에 저장된 임베딩들을 시각화해봅시다.

## t-SNE를 이용한 차원 축소

고차원 임베딩 벡터(3072차원)를 2D/3D로 축소하여 시각화합니다.

**t-SNE (t-distributed Stochastic Neighbor Embedding)란?**
- 고차원 데이터의 구조를 저차원에서 보존하는 기법
- 비슷한 데이터 포인트는 가깝게, 다른 데이터 포인트는 멀리 배치
- 클러스터링 패턴을 시각적으로 확인할 수 있음

색상 코드:
- 🔵 파란색: products (제품)
- 🟢 녹색: employees (직원)
- 🔴 빨간색: contracts (계약)
- 🟠 주황색: company (회사 정보)

In [ ]:
chroma = PersistentClient(path=DB_NAME)
collection = chroma.get_or_create_collection(collection_name)
result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
metadatas = result['metadatas']
doc_types = [metadata['type'] for metadata in metadatas]
colors = [['blue', 'green', 'red', 'orange'][['products', 'employees', 'contracts', 'company'].index(t)] for t in doc_types]

In [ ]:
tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [ ]:
tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=10, b=10, l=10, t=40)
)

fig.show()

## 고급 RAG 시스템 구축

이제 본격적으로 **고급 RAG**를 구축합니다!

### 적용할 고급 기법들:

#### 1. Reranking (재순위화)
- 벡터 유사도로 검색한 결과를 LLM이 다시 정렬
- 단순 유사도보다 **의미적 관련성**을 더 정확하게 판단
- 상위 K개 결과의 품질을 크게 향상시킴

**Reranking이 필요한 이유:**
- 임베딩 유사도는 "비슷함"을 측정하지만, "관련성"과는 다를 수 있음
- 예: "사과"와 "배"는 유사하지만, "사과 가격"을 물으면 "배 가격" 문서는 관련 없음
- LLM은 질문의 **의도**를 이해하여 더 정확한 순위 부여 가능

#### 2. Query Rewriting (쿼리 재작성)
- 사용자 질문을 검색에 최적화된 형태로 변환
- 대화 맥락을 고려한 독립적인 쿼리 생성
- 검색 정확도 향상

**Query Rewriting 예시:**
- 원본: "그 사람 어디 대학 나왔어?" 
- 재작성: "직원 학력 대학교 출신"

In [ ]:
class RankOrder(BaseModel):
    order: list[int] = Field(
        description="The order of relevance of chunks, from most relevant to least relevant, by chunk id number"
    )

In [ ]:
def rerank(question, chunks):
    system_prompt = """
You are a document re-ranker.
You are provided with a question and a list of relevant chunks of text from a query of a knowledge base.
The chunks are provided in the order they were retrieved; this should be approximately ordered by relevance, but you may be able to improve on that.
You must rank order the provided chunks by relevance to the question, with the most relevant chunk first.
Reply only with the list of ranked chunk ids, nothing else. Include all the chunk ids you are provided with, reranked.
"""
    user_prompt = f"The user has asked the following question:\n\n{question}\n\nOrder all the chunks of text by relevance to the question, from most relevant to least relevant. Include all the chunk ids you are provided with, reranked.\n\n"
    user_prompt += "Here are the chunks:\n\n"
    for index, chunk in enumerate(chunks):
        user_prompt += f"# CHUNK ID: {index + 1}:\n\n{chunk.page_content}\n\n"
    user_prompt += "Reply only with the list of ranked chunk ids, nothing else."
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    response = completion(model=MODEL, messages=messages, response_format=RankOrder)
    reply = response.choices[0].message.content
    order = RankOrder.model_validate_json(reply).order
    print(order)
    return [chunks[i - 1] for i in order]

In [ ]:
RETRIEVAL_K = 10

def fetch_context_unranked(question):
    """
    질문을 벡터로 변환하여 ChromaDB에서 유사한 문서 검색
    
    openai.embeddings.create() 반환 구조:
        CreateEmbeddingResponse(
            data=[Embedding(embedding=[0.0023, -0.009, ...], index=0)],
            model='text-embedding-3-large',
            usage=Usage(prompt_tokens=5, total_tokens=5)
        )
    
    .data[0].embedding → 질문의 의미를 표현하는 3072차원 벡터
    """
    # 질문을 벡터로 변환 (질문의 의미를 숫자로 표현)
    # 예: "Who won the IIOTY award?" → [0.0023, -0.009, 0.015, ...] 3072개 숫자
    query = openai.embeddings.create(model=embedding_model, input=[question]).data[0].embedding
    
    # ChromaDB에서 질문 벡터와 유사한 문서 벡터 검색
    # 의미가 비슷한 텍스트는 벡터 공간에서 가까이 위치함
    results = collection.query(query_embeddings=[query], n_results=RETRIEVAL_K)
    
    chunks = []
    for result in zip(results["documents"][0], results["metadatas"][0]):
        chunks.append(Result(page_content=result[0], metadata=result[1]))
    return chunks

### Reranking 테스트

이제 Reranking이 실제로 검색 품질을 개선하는지 테스트해봅시다.

**테스트 방법:**
1. 질문으로 벡터 검색 수행 (unranked)
2. 결과 순서 확인
3. Reranking 적용
4. 개선된 순서 확인

**기대 결과:**
- 질문과 직접 관련된 청크가 상위로 이동
- 단순 키워드 매칭보다 의미적 관련성이 높은 결과

In [ ]:
question = "Who won the IIOTY award?"
chunks = fetch_context_unranked(question)

In [ ]:
for chunk in chunks:
    print(chunk.page_content[:15]+"...")

In [ ]:
reranked = rerank(question, chunks)

In [ ]:
for chunk in reranked:
    print(chunk.page_content[:15]+"...")

In [ ]:
question = "Who went to Manchester University?"
RETRIEVAL_K = 20
chunks = fetch_context_unranked(question)
for index, c in enumerate(chunks):
    if "manchester" in c.page_content.lower():
        print(index)

In [ ]:
reranked = rerank(question, chunks)

In [ ]:
for index, c in enumerate(reranked):
    if "manchester" in c.page_content.lower():
        print(index)

In [ ]:
reranked[0].page_content

In [ ]:
def fetch_context(question):
    chunks = fetch_context_unranked(question)
    return rerank(question, chunks)

In [ ]:
SYSTEM_PROMPT = """
You are a knowledgeable, friendly assistant representing the company Insurellm.
You are chatting with a user about Insurellm.
Your answer will be evaluated for accuracy, relevance and completeness, so make sure it only answers the question and fully answers it.
If you don't know the answer, say so.
For context, here are specific extracts from the Knowledge Base that might be directly relevant to the user's question:
{context}

With this context, please answer the user's question. Be accurate, relevant and complete.
"""

In [ ]:
# RAG 메시지 생성 함수
# 컨텍스트에 각 청크의 출처(source)를 포함하여 투명성 확보

def make_rag_messages(question, history, chunks):
    context = "\n\n".join(f"Extract from {chunk.metadata['source']}:\n{chunk.page_content}" for chunk in chunks)
    system_prompt = SYSTEM_PROMPT.format(context=context)
    return [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": question}]

In [ ]:
def rewrite_query(question, history=[]):
    """Rewrite the user's question to be a more specific question that is more likely to surface relevant content in the Knowledge Base."""
    message = f"""
You are in a conversation with a user, answering questions about the company Insurellm.
You are about to look up information in a Knowledge Base to answer the user's question.

This is the history of your conversation so far with the user:
{history}

And this is the user's current question:
{question}

Respond only with a single, refined question that you will use to search the Knowledge Base.
It should be a VERY short specific question most likely to surface content. Focus on the question details.
Don't mention the company name unless it's a general question about the company.
IMPORTANT: Respond ONLY with the knowledgebase query, nothing else.
"""
    response = completion(model=MODEL, messages=[{"role": "system", "content": message}])
    return response.choices[0].message.content

In [ ]:
rewrite_query("Who won the IIOTY award?", [])

### Query Rewriting의 주의점

Query Rewriting은 **양날의 검**입니다. 잘 작동하면 검색 품질이 향상되지만, 잘못되면 오히려 정확도가 떨어질 수 있습니다.

**정확성을 떨어뜨리는 경우:**

| 문제 | 예시 |
|-----|------|
| 고유명사 손실 | "IIOTY 상" → "회사 수상 내역" (IIOTY가 사라짐) |
| 과도한 일반화 | "Manchester University 출신" → "직원 학력" |
| 의도 오해 | "가격이 얼마야?" → "제품 정보" (가격 키워드 누락) |
| 환각 추가 | 없는 정보를 LLM이 추가 |
| 맥락 오해석 | 대화 히스토리를 잘못 해석 |

**해결 방법들:**

1. **Multi-query RAG**: 원본 + 재작성 쿼리 둘 다로 검색
2. **조건부 적용**: 단순 질문은 재작성 건너뛰기
3. **품질 검증**: 재작성 결과가 원본과 너무 다르면 원본 사용
4. **Fallback**: 검색 결과가 부실하면 원본으로 재시도
5. **Query Expansion + Reranking 조합**: 원본 쿼리와 재작성된 쿼리 모두로 검색하여 청크를 수집한 뒤, 전체 결과를 Reranking으로 재정렬

```
┌─────────────────────────────────────────────────────────┐
│  Query Expansion + Reranking 파이프라인                  │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  원본 쿼리: "IIOTY 상 수상자?"                           │
│       ↓                                                 │
│  ┌─────────────┐     ┌─────────────┐                   │
│  │ 원본 쿼리로  │     │ 재작성 쿼리로│                   │
│  │ 검색        │     │ 검색        │                   │
│  │ → 청크 A,B,C│     │ → 청크 D,E,F│                   │
│  └─────────────┘     └─────────────┘                   │
│       ↓                    ↓                           │
│       └────────┬───────────┘                           │
│                ↓                                        │
│         중복 제거 후 병합                                │
│         [A, B, C, D, E, F]                              │
│                ↓                                        │
│         Reranking (LLM)                                 │
│                ↓                                        │
│         최종 결과: [D, A, C, ...]                        │
│                                                         │
└─────────────────────────────────────────────────────────┘
```

이 방식은 원본 쿼리의 키워드 정확성과 재작성 쿼리의 의미 확장을 모두 활용할 수 있습니다.

실무에서는 Query Rewriting을 **선택적으로** 적용하거나, A/B 테스트로 효과를 검증합니다.

In [ ]:
def answer_question(question: str, history: list[dict] = []) -> tuple[str, list]:
    """
    Answer a question using RAG and return the answer and the retrieved context
    """
    query = rewrite_query(question, history)
    print(query)
    chunks = fetch_context(query)
    messages = make_rag_messages(question, history, chunks)
    response = completion(model=MODEL, messages=messages)
    return response.choices[0].message.content, chunks

In [ ]:
answer_question("Who won the IIOTY award?", [])

In [ ]:
answer_question("Who went to Manchester University?", [])

## 정리

이번 노트북에서 학습한 고급 RAG 기법들:

### 1. LLM 기반 지능형 청킹
- 고정 크기 분할 대신 LLM이 의미 단위로 분할
- headline + summary + original_text 구조로 검색 품질 향상
- Pydantic을 활용한 Structured Output

### 2. Reranking (재순위화)
- 벡터 유사도 검색 후 LLM이 재정렬
- 의미적 관련성 기반 순위 부여
- 검색 정확도 대폭 향상

### 3. Query Rewriting (쿼리 재작성)
- 사용자 질문을 검색 최적화 형태로 변환
- 대화 맥락 고려
- 검색 정확도 향상

### 다음 단계
- HyDE (Hypothetical Document Embeddings)
- Multi-query RAG
- RAG Fusion
- Self-RAG